In [1]:
!pip install pyspark==3.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.2 MB/s eta 0:00:0000:0100:02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845553 sha256=382cfa9b3100052e08c3c2b15cd1b6b854474fe66987bad257ffd0f50c450888
  Stored in directory: /home/dev/.cache/pip/wheels/51/c8/18/298a4ced8ebb3ab8a7d26a7198c0cc7035abb906bde94a4c4b
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


25/01/18 15:05:32 WARN Utils: Your hostname, ANDERSON resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/01/18 15:05:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/18 15:05:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.json('../../curso2/datalake/twitter_datascience')

In [5]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_date|
+--------------------+--------------------+------------------+------------+
|[{19, 32, 2025-01...|{[{2025-01-14T21:...|{1234567890abcdef}|  2025-01-14|
|[{37, 42, 2025-01...|{[{2025-01-14T05:...|              null|  2025-01-14|
|[{77, 90, 2025-01...|{[{2025-01-16T18:...|              null|  2025-01-16|
|[{55, 51, 2025-01...|{[{2025-01-12T23:...|              null|  2025-01-12|
|[{19, 87, 2025-01...|{[{2025-01-17T10:...|              null|  2025-01-17|
|[{43, 86, 2025-01...|{[{2025-01-13T20:...|              null|  2025-01-13|
|[{2, 66, 2025-01-...|{[{2025-01-15T03:...|              null|  2025-01-15|
|[{58, 46, 2025-01...|{[{2025-01-11T20:...|              null|  2025-01-11|
|[{17, 2, 2025-01-...|{[{2025-01-10T08:...|              null|  2025-01-10|
+--------------------+--------------------+------------------+------------+



In [6]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c